This script serves the building of a Heliophysics dictionary. 

As we make contributions to the Unified Astronomy Thesaurus, this script will take as input those contributed terms, attach to them the existing definitions from a compendium of Heliophysics terms lists/dictionaries and output them to a spreadsheet for harmonization and contribution to (an authoritative) Helio dictionary. 

Input: a Heliophysics term
Process: 
    look through the following glossaries for definitions and pull out all that exist:

    - HELIO Ontology
    - Heliophysics Event Knowledge Base
    - NASA CCMC
    - SWEET
    - SPASE
    - NASA Heliophysics Vocabulary
    - Space Weather Glossary
    - Space Weather Glossary (Second)
    - ESA Space Weather Glossary (ESA)

    (not yet applied:) identify similar terms to the input term


Output: {term: definition1, definition2, etc.}



NOTE this builds on 'add_definitions_to_terms.ipynb' 


In [2]:
import os, sys, re
import pandas as pd
import numpy as np

from glob import glob
import json


import csv


from rdflib import Graph, Namespace, RDFS, URIRef, Literal
from collections import defaultdict
import difflib

from difflib import SequenceMatcher

In [3]:
# Functions to populate a Helio dictionary harmonization file V1

def update_csv(csv_filename, term, column_names, entries):
    # Ensure the CSV exists and has the right columns
    file_exists = os.path.isfile(csv_filename)
    
    # Always include 'term' as the first column
    full_columns = ['term'] + column_names
    

    # Check for existing term
    if file_exists:
        with open(csv_filename, mode='r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row.get('term', '').strip() == term:
                    print(f"Term '{term}' already exists in CSV. Skipping row addition.")
                    return  # Don't add duplicate term
                
    if not file_exists:
        # Create CSV with header
        with open(csv_filename, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=full_columns)
            writer.writeheader()
        

    # Prepare a new row with empty cells
    new_row = {col: '' for col in full_columns}
    new_row['term'] = term  # Set the term value
    
    # Fill in the appropriate columns from the entries
    for entry in entries:
        if 'source:' in entry:
            description, source_info = entry.split('source:', 1)
            description = description.strip()
            source_file = source_info.strip()
            
            # Match the column based on the source filename
            matched = False
            for col in column_names:
                if source_file.startswith(col):
                    new_row[col] = description
                    matched = True
                    break
            if not matched:
                print(f"Warning: No matching column found for source '{source_file}'.")
    
    # Append the new row
    with open(csv_filename, mode='a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=full_columns)
        writer.writerow(new_row)

In [4]:
# Functions to populate a Helio dictionary harmonization file V2

# List of fixed source columns
SOURCES = [
    'HELIO Ontology.json',
    'Heliophysics Events Knowledgebase.json',
    'NASA CCMC.json',
    'Semantic Web for Earth and Environment Technology Ontology.json',
    'SPASE Dictionary.json',
    'NASA Heliophysics Vocabulary.json',
    'Space Weather Glossary.json',
    'Space Weather Glossary (Second).json',
    'ESA Space Weather Glossary.json'
]

# CSV file path for running output
OUTPUT_FILE = "/Users/ryanmc/Documents/Helio_ECIP/dev/Helio-KNOW/ADS_enrichment/TEST_Heliophysics_dictionary_harmonization.csv"

def fuzz_ratio(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def add_term_to_definition_matrix(input_term, df, output_file=OUTPUT_FILE, threshold=0.6):
    # Prepare one output row
    output_row = {"term": input_term}

    for source in SOURCES:
        subset = df[df['type'] == source]

        # Exact match
        exact_match = subset[subset['term'].str.lower() == input_term.lower()]
        if not exact_match.empty:
            output_row[source] = subset['term'].str.lower() + str(': ') + exact_match.iloc[0]['definition']
        else:
            # Fuzzy match
            subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
            top_match = subset[subset['score'] >= threshold].sort_values(by='score', ascending=False)
            if not top_match.empty:
                output_row[source] = subset['term'].str.lower() + str(': ') + top_match.iloc[0]['definition']
            else:
                output_row[source] = ""

    # Append to or create the CSV
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        if input_term not in existing_df['term'].values:
            existing_df = pd.concat([existing_df, pd.DataFrame([output_row])], ignore_index=True)
            existing_df.to_csv(output_file, index=False)
    else:
        new_df = pd.DataFrame([output_row], columns=["term"] + SOURCES)
        new_df.to_csv(output_file, index=False)

    return output_row


In [5]:
list_glossaries = ['HELIO Ontology',
                   'Heliophysics Event Knowledge Base',
                   'NASA CCMC',
                   'SWEET',
                   'SPASE',
                   'NASA Heliophysics Vocabulary',
                   'Space Weather Glossary',
                   'Space Weather Glossary (Second)',
                   'ESA Space Weather Glossary (ESA)']

In [6]:
glossaries_list = [
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/HELIO Ontology.json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/Heliophysics Events Knowledgebase.json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/NASA CCMC.json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/Semantic Web for Earth and Environment Technology Ontology.json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/SPASE Dictionary.json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/NASA Heliophysics Vocabulary.json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/Space Weather Glossary.json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/Space Weather Glossary (Second).json',
     '/Users/ryanmc/Documents/NASA_FDL/2022/sdm-kg-nasa-2022/data/raw/SMDVocabulary/SupersetVocab/ESA Space Weather Glossary.json',
]


pd_compiled = pd.DataFrame(columns=['term','definition','source'])

column_names = []

terms_full = []
definitions_full = []
sources_full = []

for g in glossaries_list:#[0:1]:
    column_names.append(g[92:])
    print('working on {}...'.format(column_names[-1]))
    
    
    with open(g) as json_file:
        content = json.load(json_file)
        terms_loop = []
        definitions_loop = []
        sources_loop = []
        for inx, term in enumerate(content["Terms"]):
            terms_loop.append(re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', term['Term']).lower())
            definitions_loop.append(term['Definition'])
        sources_loop = list(np.tile(str(g[92:]),(len(terms_loop),1)).flatten())
    
    # Add to full lists
    terms_full = terms_full + terms_loop
    definitions_full = definitions_full + definitions_loop
    sources_full = sources_full + sources_loop
    
pd_full = pd.DataFrame({'term':terms_full,'definition':definitions_full,'type':sources_full})
pd_full

working on HELIO Ontology.json...
working on Heliophysics Events Knowledgebase.json...
working on NASA CCMC.json...
working on Semantic Web for Earth and Environment Technology Ontology.json...
working on SPASE Dictionary.json...
working on NASA Heliophysics Vocabulary.json...
working on Space Weather Glossary.json...
working on Space Weather Glossary (Second).json...
working on ESA Space Weather Glossary.json...


,term,definition,type
0,thing,,HELIO Ontology.json
1,data resource,,HELIO Ontology.json
2,service,,HELIO Ontology.json
3,catalog,A tabular listing of events or observational n...,HELIO Ontology.json
4,record,,HELIO Ontology.json
...,...,...,...
14945,spacecraft effects,Effects observed as a result of the interactio...,ESA Space Weather Glossary.json
14946,space weather,Space Weather is the physical and phenomenolog...,ESA Space Weather Glossary.json
14947,space weather event,A time-limited condition of the space environm...,ESA Space Weather Glossary.json
14948,susceptibility,The response of a material or substance to a c...,ESA Space Weather Glossary.json


In [9]:
#INPUT a term
term = input('please enter a term to get defintions for: ')

please enter a term to get defintions for: magnetosphere


In [10]:
# Exact match
idx = np.argwhere(pd_full['term'].values == term)

In [11]:
defs_loop = []
for i in idx:
#     print(pd_full.iloc[i])
    if pd_full.iloc[i]['definition'].values:
        defs_loop.append( pd_full.iloc[i]['definition'].values[0] + str('    source: ').format() + pd_full.iloc[i]['type'].values[0])


In [12]:
defs_loop

['The region of space dominated by the magnetic field of a star or planet. Earth’s magnetosphere takes on a tear-drop shape under the influence of the flowing solar wind.    source: NASA CCMC.json',
 "The region of space above the atmosphere orsurface of the planet, and bounded by the magnetopause, that is under thedirect influence of the planet's magnetic field.    source: SPASE Dictionary.json",
 'The magnetosphere is a bubble of magnetic fields that surrounds Earth, created by the natural magnetism of the planet. The magnetosphere protects humans on Earth from incoming energy from the sun, however it does change shape and size in response to such space weather, and these fluctuations can degrade communication signals and cause unexpected electrical surges in power grids.    source: NASA Heliophysics Vocabulary.json',
 'The magnetic cavity surrounding a magnetized body, carved out of the passing solar wind by virtue of the magnetic field, which prevents, or at least impedes, the dire

In [13]:
# NOT YET APPLIED: identify similar terms and attach their definitions with the similar term for later work in harmonizing

# fuzzy match
close_terms = difflib.get_close_matches(term, pd_full['term'], n=40, cutoff=0.6)
close_terms

['magnetosphere',
 'magnetosphere',
 'magnetosphere',
 'magnetosphere',
 'magnetosphere',
 'magnetosphere',
 'mesosphere',
 'mesosphere',
 'mesosphere',
 'mesosphere',
 'mesosphere',
 'atmosphere',
 'atmosphere',
 'magnetosheath',
 'magnetosheath',
 'magnetosheath',
 'magnetosheath',
 'magnetosheath',
 'exosphere',
 'exosphere',
 'exosphere',
 'exosphere',
 'planetary magnetosphere',
 'stratosphere',
 'stratosphere',
 'stratosphere',
 'stratosphere',
 'stratosphere',
 'magnetopause',
 'magnetopause',
 'magnetopause',
 'magnetopause',
 'magnetometer',
 'magnetometer',
 'magnetometer',
 'magnetograph',
 'magnetograph',
 'ionosphere',
 'ionosphere',
 'ionosphere']

In [26]:
column_names

['HELIO Ontology.json',
 'Heliophysics Events Knowledgebase.json',
 'NASA CCMC.json',
 'Semantic Web for Earth and Environment Technology Ontology.json',
 'SPASE Dictionary.json',
 'NASA Heliophysics Vocabulary.json',
 'Space Weather Glossary.json',
 'Space Weather Glossary (Second).json',
 'ESA Space Weather Glossary.json']

In [34]:
# output to csv

# Example usage
csv_name = '/Users/ryanmc/Documents/Helio_ECIP/dev/Helio-KNOW/ADS_enrichment/Heliophysics_dictionary_harmonization.csv'

update_csv(csv_name, term, column_names, defs_loop)


Term 'aurora' already exists in CSV. Skipping row addition.


Set up a loop that goes through all entities in the region-based taxonomy and runs the code above (will later add the phenomena ontology for populating the Helio dictionary, but these will not be contributed to the UAT)


OUTPUT will be: 
- all terms added to Helio dictionary harmonizer with available definitions
- all terms have an output csv file for use in making request for contribution to UAT, including the most similar terms and if there is an identical term, only that term identified (meaning it is already there and we need not do a concept request)


In [14]:
# Fuzzy similarity using difflib
def fuzz_ratio(a, b):
    return int(100 * difflib.SequenceMatcher(None, a, b).ratio())

# Load TTL and extract labels and parent links
def load_taxonomy(file_path):
    g = Graph()
    g.parse(file_path, format="ttl")
    SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")

    labels = {}
    parents = defaultdict(list)

    for s, p, o in g:
        s_str = str(s)
        if p in (SKOS.prefLabel, RDFS.label) and isinstance(o, Literal):
            labels[s_str] = str(o)
        if p in (SKOS.broader, RDFS.subClassOf) and isinstance(o, URIRef):
            parents[s_str].append(str(o))
    return labels, parents

# Build paths from term to root
def build_paths(parents):
    paths = {}

    def get_path(term, seen=None):
        if seen is None:
            seen = set()
        if term in seen:
            return []
        seen.add(term)
        if term not in parents:
            return [term]
        all_paths = []
        for parent in parents[term]:
            path = get_path(parent, seen.copy())
            all_paths.append(path + [term])
        return max(all_paths, key=len) if all_paths else [term]

    for term in parents:
        paths[term] = get_path(term)
    return paths


# Updated function to write match results per term to separate CSVs
def write_matches_to_csv_per_term(labels1, paths1, labels2, paths2, helio_dictionary_name, definitions_df, output_dir="/Users/ryanmc/Documents/Helio_ECIP/data/UAT_comparison_terms_output/", threshold=50):
    os.makedirs(output_dir, exist_ok=True)
    csv_files = []

    for term1_uri, term1_label in labels1.items():
        print('-------------------------\n \tWorking on term = {}\n\n'.format(term1_label))
        term1_label_loop = term1_label.replace("/", " ").replace("-", " ")
        
        path1 = " > ".join(labels1.get(t, t) for t in paths1.get(term1_uri, []))

        filename = f"{term1_label_loop.strip().replace(' ', '_')}_uat_matches.csv"
        filepath = os.path.join(output_dir, filename)
        csv_files.append(filepath)
        
        exact_matches = []
        fuzzy_matches = []
        
        
        
        for term2_uri, term2_label in labels2.items():
            path2 = " > ".join(labels2.get(t, t) for t in paths2.get(term2_uri, []))

            if term1_label_loop.strip().lower() == term2_label.strip().lower():
                exact_matches.append({
                    "term1_uri": term1_uri,
                    "term1_label": term1_label_loop,
                    "term2_uri": term2_uri,
                    "term2_label": term2_label,
                    "label_score": 100,
                    "path_score": fuzz_ratio(path1, path2),
                    "final_score": 100
                })
            else:
                label_score = fuzz_ratio(term1_label_loop, term2_label)
                path_score = fuzz_ratio(path1, path2)
                final_score = 0.7 * label_score + 0.3 * path_score

                if final_score >= threshold:
                    fuzzy_matches.append({
                        "term1_uri": term1_uri,
                        "term1_label": term1_label_loop,
                        "term2_uri": term2_uri,
                        "term2_label": term2_label,
                        "label_score": label_score,
                        "path_score": path_score,
                        "final_score": final_score
                    })

        if exact_matches:
            matches = exact_matches
        elif fuzzy_matches:
            matches = sorted(fuzzy_matches, key=lambda x: -x["final_score"])
        else:
            with open(filepath, mode="w", newline='', encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=[
                    "term1_uri", "term1_label", "term2_uri", "term2_label",
                    "label_score", "path_score", "final_score"
                ])
                writer.writeheader()
            continue  # No match above threshold



        with open(filepath, mode="w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=[
                "term1_uri", "term1_label", "term2_uri", "term2_label",
                "label_score", "path_score", "final_score"
            ])
            writer.writeheader()
            for match in matches:
                writer.writerow(match)
                
        Finally, compile definitions for the term and add to Helio harmonization dictionary
        # For exact match
        idx = np.argwhere(definitions_df['term'].values == term1_label_loop)
        defs_loop = []
        for i in idx:
            if definitions_df.iloc[i]['definition'].values:
                defs_loop.append( definitions_df.iloc[i]['definition'].values[0] + str('    source: ').format() + definitions_df.iloc[i]['type'].values[0])
        # NOT YET APPLIED: identify similar terms and attach their definitions with the similar term for later work in harmonizing

        update_csv(helio_dictionary_name, term1_label, column_names, defs_loop)
        
        add_term_to_definition_matrix(term1_label, pd_full)

    return csv_files

In [15]:
# Run the code 

helio_dictionary_name = '/Users/ryanmc/Documents/Helio_ECIP/dev/Helio-KNOW/ADS_enrichment/Heliophysics_dictionary_harmonization_v2.csv'

file1 = "/Users/ryanmc/Documents/Helio_ECIP/dev/Helio-KNOW/data-models/hk_region.ttl"
file2 = "/Users/ryanmc/Documents/Helio_ECIP/dev/Helio-KNOW/ADS_enrichment/uat.ttl"

labels1, parents1 = load_taxonomy(file1)
labels2, parents2 = load_taxonomy(file2)

paths1 = build_paths(parents1)
paths2 = build_paths(parents2)


output_csvs = write_matches_to_csv_per_term(labels1, paths1, labels2, paths2, helio_dictionary_name, pd_full)
# output_csvs[:5]  # Show first few output files generated

-------------------------
 	Working on term = Stratosphere




/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarn

-------------------------
 	Working on term = D Region




/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarn

-------------------------
 	Working on term = Ground and Lower Atmosphere




...

/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarn

-------------------------
 	Working on term = F Region




/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarn

-------------------------
 	Working on term = Bowshock




/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_48230/2245894202.py:35: SettingWithCopyWarn

In [84]:
label_test = 'Phenomenon'

idx = np.argwhere(pd_full['term'].values == label_test)
print('exact matches {}'.format(idx))

close_terms = difflib.get_close_matches(label_test, pd_full['term'], n=40, cutoff=0.7)
print('close matches {}'.format(close_terms))
close_terms

defs_loop = []
for i in idx:
    if definitions_df.iloc[i]['definition'].values:
        defs_loop.append( term1_label + str(': ') + definitions_df.iloc[i]['definition'].values[0] + str('    (source: ').format() + definitions_df.iloc[i]['type'].values[0] + str(')'))
# NOT YET APPLIED: identify similar terms and attach their definitions with the similar term for later work in harmonizing

for c in close_terms:
    if definitions_df.iloc[i]['definition'].values:


exact matches []
close matches ['phenomenon', 'phenomenon', 'phenomena', 'phenomenon type']


['phenomenon', 'phenomenon', 'phenomena', 'phenomenon type']

/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_60679/606415245.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_60679/606415245.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['score'] = subset['term'].apply(lambda t: fuzz_ratio(input_term, t))
/var/folders/56/vhtxcmr565z5ksdt3g3px27w0000gq/T/ipykernel_60679/606415245.py:33: SettingWithCopyWarning

{'term': 'Solar Wind',
 'HELIO Ontology.json': 0               thing: 
 1       data resource: 
 2             service: 
 3             catalog: 
 4              record: 
              ...       
 1124        hatimeend: 
 1125       hatimepeak: 
 1126      hatimestart: 
 1127      haxrayclass: 
 1128      hessi_flare: 
 Name: term, Length: 1129, dtype: object,
 'Heliophysics Events Knowledgebase.json': '',
 'NASA CCMC.json': 1370    ambient solar wind: plasma flowing out from th...
 1371                 ace: plasma flowing out from the sun
 1372      active regions: plasma flowing out from the sun
 1373    ambient solar wind: plasma flowing out from th...
 1374                  au: plasma flowing out from the sun
                               ...                        
 1443            streamer: plasma flowing out from the sun
 1444             sunspot: plasma flowing out from the sun
 1445          tachocline: plasma flowing out from the sun
 1446    transition region: plasma flowin

In [ ]:
idx = np.argwhere(definitions_df['term'].values == 'Phenomenon')
defs_loop = []
for i in idx:
    if definitions_df.iloc[i]['definition'].values:
        defs_loop.append( term1_label + str(': ') + definitions_df.iloc[i]['definition'].values[0] + str('    (source: ').format() + definitions_df.iloc[i]['type'].values[0] + str(')'))
# NOT YET APPLIED: identify similar terms and attach their definitions with the similar term for later work in harmonizing


update_csv(helio_dictionary_name, term1_label, column_names, defs_loop)